In [1]:
# 1. import any required library to load dataset, open files (os), print confusion matrix and accuracy score
import pickle
import pandas as pd
import os
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
# 2. Create test set if you like to do the split programmatically or if you have not already split the data at this point

# Load dataset
df = pd.read_csv('disadvantaged_communities.csv')

#encode categorical variables...
from category_encoders import OrdinalEncoder
# Define categorical columns to be encoded
categorical_cols = df.columns.drop(['CES 4.0 Percentile Range'])  # append all cols in list except target variable column
# Create an instance of OrdinalEncoder
encoder = OrdinalEncoder(cols=categorical_cols)
# Fit and transform the categorical variables in the DataFrame
df_encoded = encoder.fit_transform(df)

#split data (90% 10% split for evaluation purpose.  only going to use the 10%)

# Separate predictor variables from the target variable (attributes (X) and target variable (y) as we did in the class)
X = df_encoded.drop(columns=['CES 4.0 Percentile Range'])
# Select the target column
y = df_encoded['CES 4.0 Percentile Range']  # All rows, column 9


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)

C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtyp

In [3]:
# 3. Load your saved model for dissadvantaged communities classification 
#that you saved in dissadvantaged_communities_classification.py via Pikcle

# Load the models from the file
with open('SvmClassifier.sav', 'rb') as file:
    svm = pickle.load(file)

with open('RfClassifier.sav', 'rb') as file:
    Rf = pickle.load(file)


In [ ]:
# 5. use predictions and test_set (X_test) classifications to print the following:
#    1. confution matrix, 2. accuracy score, 3. precision, 4. recall, 5. specificity
#    You can easily find the formulae for Precision, Recall, and Specificity online.

# Get and print confusion matrix
cm = [[]]
# Below are the metrics for computing classification accuracy, precision, recall and specificity
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]